### Imports necessary libraries for data manipulation, visualization, model training, and evaluation.

In [1]:
!pip install scikit-learn==1.2.2

DEPRECATION: Loading egg at c:\users\dodiy\anaconda3\lib\site-packages\diamondpriceprediction-0.0.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at c:\users\dodiy\anaconda3\lib\site-packages\fonttools-4.44.0-py3.11-win-amd64.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn.model_selection import train_test_split , cross_val_score , GridSearchCV , RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder , OneHotEncoder , StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBRFClassifier , XGBClassifier
from sklearn.ensemble import VotingClassifier , RandomForestClassifier , GradientBoostingClassifier
from sklearn.metrics import confusion_matrix , accuracy_score , classification_report
from sklearn.pipeline import make_pipeline , Pipeline
from sklearn.compose import ColumnTransformer , make_column_transformer

### Reads a CSV file ('Popular_Baby_Names.csv') into a DataFrame and displays the first few rows.

In [3]:
names = ['year_of_birth' , 'gender' , 'ethnicity' , 'child_first_name' , 'count' , 'rank']
df = pd.read_csv('Popular_Baby_Names.csv' , names=names , header=0)
df.head()

,year_of_birth,gender,ethnicity,child_first_name,count,rank
0,2011,FEMALE,HISPANIC,GERALDINE,13,75
1,2011,FEMALE,HISPANIC,GIA,21,67
2,2011,FEMALE,HISPANIC,GIANNA,49,42
3,2011,FEMALE,HISPANIC,GISELLE,38,51
4,2011,FEMALE,HISPANIC,GRACE,36,53


In [4]:
df['rank'].unique()

array([ 75,  67,  42,  51,  53,  62,   8,  74,  71,  78,  73,  72,  77,
        60,   1,  70,  61,  65,  58,  41,  76,  68,  48,  55,  40,  66,
        64,  34,  44,  57,  63,  33,  31,  20,  69,  35,  13,  52,  59,
        39,   9,  27,  10,  56,  12,   2,  25,  18,  14,  38,  28,   6,
         3,  19,  45,  47,  11,  79,  17,  43,  80,  37,  81,  46,   5,
        22,  50,  21,  30,  24,  54,  15,  36,  23,   7,  16,  49,  29,
         4,  32,  26,  92,  90,  82,  91,  88,  89,  94,  83,  93,  84,
        87,  85,  86,  96,  97,  95,  99,  98, 100, 101, 102], dtype=int64)

In [5]:
df['rank'].nunique()

102

- df['rank'] = 1 to 25 = high_popular
- df['rank'] = 25 to 50 = popular
- df['rank'] = 50 to 75 = medium_popular
- df['rank'] = 75 to _ = low_popular

### Updates the 'target' column in the DataFrame based on specified rank ranges.

In [6]:
df['target'] = 'less_popular'

# Update 'target' based on the specified ranges
df.loc[(df['rank'] >= 1) & (df['rank'] <= 25), 'target'] = 'high_popular'
df.loc[(df['rank'] > 25) & (df['rank'] <= 50), 'target'] = 'popular'
df.loc[(df['rank'] > 50) & (df['rank'] <= 75), 'target'] = 'medium_popular'

In [7]:
df['target'].unique()

array(['medium_popular', 'popular', 'high_popular', 'less_popular'],
      dtype=object)

In [8]:
df['target'].value_counts()

target
less_popular      17373
medium_popular    16806
popular           15941
high_popular       7462
Name: count, dtype: int64

In [9]:
mapping = {
    "high_popular" : 0,
    "popular" : 1,
    "medium_popular" : 2,
    "less_popular" : 3
}
df['target'] = df['target'].map(mapping)
df['target'].value_counts()

target
3    17373
2    16806
1    15941
0     7462
Name: count, dtype: int64

In [10]:
df.head()

,year_of_birth,gender,ethnicity,child_first_name,count,rank,target
0,2011,FEMALE,HISPANIC,GERALDINE,13,75,2
1,2011,FEMALE,HISPANIC,GIA,21,67,2
2,2011,FEMALE,HISPANIC,GIANNA,49,42,1
3,2011,FEMALE,HISPANIC,GISELLE,38,51,2
4,2011,FEMALE,HISPANIC,GRACE,36,53,2


In [11]:
X = df.drop(['rank','target'] , axis=1)
y = df['target']

X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [12]:
nominal_cols = ['gender','ethnicity']
label_col = ['child_first_name']
standard_scaling_data = ['year_of_birth', 'gender', 'ethnicity', 'child_first_name', 'count']

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder

#label_encode_cols = ['child_first_name']
one_hot_encode_cols = ['gender', 'ethnicity','child_first_name']
scale_cols = ['count']

preprocessor = ColumnTransformer(
    transformers=[
        ('one_hot_encode', OneHotEncoder(drop='first',handle_unknown='ignore' , sparse_output=False), one_hot_encode_cols),
        ('scale', StandardScaler(), ['count']),
    ],
    remainder='passthrough'
)

# Create the final pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

In [14]:
X_train_transformed = pipeline.fit_transform(X_train)
column_names = (pipeline.named_steps['preprocessor']
                .named_transformers_['one_hot_encode']
                .get_feature_names_out(input_features=one_hot_encode_cols))
column_names = list(column_names) + ['count'] + ['year_of_birth']

X_train_transformed_df = pd.DataFrame(X_train_transformed,columns=column_names)
X_train_transformed_df.head()

,gender_MALE,ethnicity_ASIAN AND PACIFIC ISLANDER,ethnicity_BLACK NON HISP,ethnicity_BLACK NON HISPANIC,ethnicity_HISPANIC,ethnicity_WHITE NON HISP,ethnicity_WHITE NON HISPANIC,child_first_name_AARAV,child_first_name_AARON,child_first_name_AAYAN,...,child_first_name_Zion,child_first_name_Zissy,child_first_name_Zoe,child_first_name_Zoey,child_first_name_Zora,child_first_name_Zoya,child_first_name_Zuri,child_first_name_Zyaire,count,year_of_birth
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.005663,2011.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.773576,2012.0
2,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.414455,2011.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.483347,2019.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.260954,2012.0


In [15]:
X_test_transformed = pipeline.transform(X_test)
column_names = (pipeline.named_steps['preprocessor']
                .named_transformers_['one_hot_encode']
                .get_feature_names_out(input_features=one_hot_encode_cols))
column_names = list(column_names) + ['count'] + ['year_of_birth']

X_test_transformed_df = pd.DataFrame(X_test_transformed, columns=column_names)
X_test_transformed_df.head()

c:\Users\dodiy\Documents\100_projects\popular_baby_names\.venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


,gender_MALE,ethnicity_ASIAN AND PACIFIC ISLANDER,ethnicity_BLACK NON HISP,ethnicity_BLACK NON HISPANIC,ethnicity_HISPANIC,ethnicity_WHITE NON HISP,ethnicity_WHITE NON HISPANIC,child_first_name_AARAV,child_first_name_AARON,child_first_name_AAYAN,...,child_first_name_Zion,child_first_name_Zissy,child_first_name_Zoe,child_first_name_Zoey,child_first_name_Zora,child_first_name_Zoya,child_first_name_Zuri,child_first_name_Zyaire,count,year_of_birth
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.570404,2011.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.355089,2012.0
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.953544,2015.0
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.252483,2017.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.534650,2017.0


In [54]:
model_list = {
    "LogisticRegression" : LogisticRegression(),
    "DecisionTreeClassifier" : DecisionTreeClassifier(),
    "XGBClassifier" : XGBClassifier()
}

In [55]:
model_list.items()

dict_items([('LogisticRegression', LogisticRegression()), ('DecisionTreeClassifier', DecisionTreeClassifier()), ('XGBClassifier', XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...))])

In [56]:
for name , model in model_list.items():

  model = model.fit(X_train_transformed_df,y_train)
  y_pred = model.predict(X_test_transformed_df)

  print(f"Model Name = {name}")
  print(f"Accuracy Score = {accuracy_score(y_test,y_pred)}")

  print("*"*50)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model Name = LogisticRegression
Accuracy Score = 0.8384127811061909
**************************************************
Model Name = DecisionTreeClassifier
Accuracy Score = 0.9965268733177043
**************************************************
Model Name = XGBClassifier
Accuracy Score = 0.9982634366588521
**************************************************


In [62]:
model = XGBClassifier()
model.fit(X_train_transformed_df,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [17]:
import joblib

joblib.dump(model , "Xgb_model.joblib")
joblib.dump(pipeline , "preprocessor.joblib")

NameError: name 'model' is not defined

In [18]:
import joblib

In [19]:
model = joblib.load('Xgb_model.joblib')

In [20]:
model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [17]:
import joblib
joblib.dump(preprocessor , "preprocessor__.joblib")

['preprocessor__.joblib']

In [16]:
import sklearn

print("Scikit-learn version:", sklearn.__version__)


Scikit-learn version: 1.4.1.post1
